In [ ]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("📚 라이브러리 로드 완료")


In [ ]:
# 데이터 파일 읽기
df_facility = pd.read_csv('/content/drive/MyDrive/공공빅데이터분석PBL/midterm/소상공인시장진흥공단_전통시장현황_20240719.csv', encoding='euc-kr')
df_std = pd.read_csv('/content/drive/MyDrive/공공빅데이터분석PBL/midterm/전국전통시장표준데이터.csv', encoding='euc-kr')

print(f"📁 첫 번째 파일 (시설 정보): {len(df_facility)}개 행, {len(df_facility.columns)}개 컬럼")
print(f"📁 두 번째 파일 (표준 데이터): {len(df_std)}개 행, {len(df_std.columns)}개 컬럼")

print("\n🔍 첫 번째 파일 컬럼:")
for i, col in enumerate(df_facility.columns[:10]):
    print(f"   {i+1:2d}. {col}")
if len(df_facility.columns) > 10:
    print(f"   ... (총 {len(df_facility.columns)}개)")

print("\n🔍 두 번째 파일 컬럼:")
for i, col in enumerate(df_std.columns[:10]):
    print(f"   {i+1:2d}. {col}")
if len(df_std.columns) > 10:
    print(f"   ... (총 {len(df_std.columns)}개)")


In [ ]:
def clean_market_name(name):
    """시장명 정제 함수"""
    if pd.isna(name) or not name:
        return ""
    
    name = str(name)
    
    # 괄호와 그 안의 내용 제거
    name = re.sub(r'\([^)]*\)', '', name)
    
    # 특수문자 제거 (한글, 영문, 숫자, 공백만 남김)
    name = re.sub(r'[^\w\s가-힣]', '', name)
    
    # 공백 정리
    name = ' '.join(name.split())
    
    return name.strip()

def standardize_region(address):
    """지역명 표준화"""
    if pd.isna(address) or not address:
        return address
    
    address = str(address)
    # 지역명 표준화
    address = address.replace('강원특별자치도', '강원도')
    address = address.replace('전북특별자치도', '전라북도')
    
    return address

def normalize_address(address):
    """주소 정규화 - 비교를 위한 주소 정제"""
    if pd.isna(address) or not address:
        return ""
    
    address = str(address)
    # 지역명 표준화
    address = standardize_region(address)
    
    # 공백 정리
    address = re.sub(r'\s+', ' ', address).strip()
    
    # 특수문자 제거 (비교용)
    address = re.sub(r'[^\w\s가-힣]', '', address)
    
    return address

print("🔧 데이터 처리 함수 정의 완료")


In [ ]:
# 데이터 전처리
print("🔄 데이터 전처리 시작...")

# 첫 번째 파일 전처리
df_facility['시장명_정제'] = df_facility['시장명'].apply(clean_market_name)
df_facility['도로명주소_정규화'] = df_facility['도로명주소'].apply(normalize_address)
df_facility['지번주소_정규화'] = df_facility['지번주소'].apply(normalize_address)

# 두 번째 파일 전처리
df_std['시장명_정제'] = df_std['시장명'].apply(clean_market_name)
df_std['소재지도로명주소_정규화'] = df_std['소재지도로명주소'].apply(normalize_address)
df_std['소재지지번주소_정규화'] = df_std['소재지지번주소'].apply(normalize_address)

print(f"✅ 첫 번째 파일 전처리 완료: {len(df_facility)}개 시장")
print(f"✅ 두 번째 파일 전처리 완료: {len(df_std)}개 시장")

# 중복 시장명 확인
facility_name_counts = df_facility['시장명_정제'].value_counts()
std_name_counts = df_std['시장명_정제'].value_counts()

facility_duplicates = facility_name_counts[facility_name_counts > 1]
std_duplicates = std_name_counts[std_name_counts > 1]

print(f"\n📊 중복 시장명 분석:")
print(f"   - 첫 번째 파일 중복 시장명: {len(facility_duplicates)}개")
print(f"   - 두 번째 파일 중복 시장명: {len(std_duplicates)}개")

if len(facility_duplicates) > 0:
    print(f"\n   첫 번째 파일 중복 예시:")
    for name, count in facility_duplicates.head(3).items():
        print(f"     - {name}: {count}개")

if len(std_duplicates) > 0:
    print(f"\n   두 번째 파일 중복 예시:")
    for name, count in std_duplicates.head(3).items():
        print(f"     - {name}: {count}개")


In [ ]:
# 매칭 함수 정의
def find_address_match(facility_row, std_candidates):
    """주소 매칭 찾기 - 도로명주소 우선, 없으면 지번주소로 보완"""
    facility_road = facility_row['도로명주소_정규화']
    facility_jibun = facility_row['지번주소_정규화']
    
    # 1단계: 도로명주소로 매칭 시도
    if facility_road and facility_road != '0':
        for idx, std_row in std_candidates.iterrows():
            std_road = std_row['소재지도로명주소_정규화']
            if std_road and facility_road == std_road:
                return idx  # 도로명주소 일치
    
    # 2단계: 도로명주소 매칭 실패 시 지번주소로 매칭 시도
    if facility_jibun:
        for idx, std_row in std_candidates.iterrows():
            std_jibun = std_row['소재지지번주소_정규화']
            if std_jibun and facility_jibun == std_jibun:
                return idx  # 지번주소 일치
    
    return None  # 매칭 실패

print("🔧 매칭 함수 정의 완료 (True/False 방식)")


In [ ]:
# 매칭 과정 실행
print("🔗 매칭 과정 시작...")

matches = []
matched_std_indices = set()

# 1단계: 고유 시장명 매칭
print("1️⃣ 고유 시장명 매칭...")
unique_name_matches = 0

for name in df_facility['시장명_정제'].unique():
    if name == "":
        continue
        
    facility_rows = df_facility[df_facility['시장명_정제'] == name]
    std_rows = df_std[df_std['시장명_정제'] == name]
    
    # 고유한 시장명인 경우 (양쪽 모두 1개씩)
    if len(facility_rows) == 1 and len(std_rows) == 1:
        facility_idx = facility_rows.index[0]
        std_idx = std_rows.index[0]
        
        matches.append((facility_idx, std_idx, 'unique_name'))
        matched_std_indices.add(std_idx)
        unique_name_matches += 1

print(f"   ✅ 고유 시장명 매칭: {unique_name_matches}개")

# 2단계: 중복 시장명 주소 매칭
print("2️⃣ 중복 시장명 주소 매칭...")
address_matches = 0

for name in df_facility['시장명_정제'].unique():
    if name == "":
        continue
        
    facility_rows = df_facility[df_facility['시장명_정제'] == name]
    std_rows = df_std[df_std['시장명_정제'] == name]
    
    # 중복 시장명인 경우
    if len(facility_rows) > 1 or len(std_rows) > 1:
        # 아직 매칭되지 않은 std 행들만 고려
        available_std_rows = std_rows[~std_rows.index.isin(matched_std_indices)]
        
        for facility_idx, facility_row in facility_rows.iterrows():
            # 이미 매칭된 facility 행은 건너뛰기
            if any(match[0] == facility_idx for match in matches):
                continue
                
            if len(available_std_rows) > 0:
                match_idx = find_address_match(facility_row, available_std_rows)
                
                if match_idx is not None:  # 매칭이 있는 경우
                    matches.append((facility_idx, match_idx, 'address'))
                    matched_std_indices.add(match_idx)
                    # 매칭된 행을 available_std_rows에서 제거
                    available_std_rows = available_std_rows.drop(match_idx)
                    address_matches += 1

print(f"   ✅ 주소 매칭: {address_matches}개")

# 매칭 결과 요약
total_matches = len(matches)
print(f"\n📊 매칭 결과:")
print(f"   - 고유 시장명 매칭: {unique_name_matches}개")
print(f"   - 주소 기반 매칭: {address_matches}개")
print(f"   - 총 매칭: {total_matches}개")
print(f"   - 매칭률: {total_matches}/{len(df_facility)} = {total_matches/len(df_facility)*100:.1f}%")


In [ ]:
# 최종 데이터 생성
print("📊 최종 데이터 생성...")

# 최종 컬럼 정의
final_columns = [
    '시장코드', '시장명', '지번주소', '도로명주소', '시도', '시군구',
    '시장개설주기', '점포수', '개설연도', '취급품목',
    # 시설 보유여부 (첫 번째 파일에서)
    '아케이드', '엘리베이터/에스컬레이터', '고객지원센터', '스프링쿨러', '화재감지기',
    '유아놀이방', '종합콜센터', '고객휴게실', '수유센터', '물품보관함',
    '자전거보관함', '체육시설', '간이도서관', '쇼핑카트', '외국인안내센터',
    '고객동선통로', '방송센터', '문화교실', '공동물류창고', '시장전용고객주차장',
    '교육장', '회의실', '자동심장충격기',
    # 시설 보유여부 (두 번째 파일에서)
    '공중화장실', '주차장',
    # 위치 및 연락처
    '위도', '경도', '전화번호'
]

# 시설 컬럼 매핑
facility_mapping = {
    '아케이드': '아케이드 보유 여부',
    '엘리베이터/에스컬레이터': '엘리베이터_에스컬레이터_보유여부',
    '고객지원센터': '고객지원센터 보유 여부',
    '스프링쿨러': '스프링쿨러 보유 여부',
    '화재감지기': '화재감지기 보유여부',
    '유아놀이방': '유아놀이방_보유여부',
    '종합콜센터': '종합콜센터_보유여부',
    '고객휴게실': '고객휴게실_보유여부',
    '수유센터': '수유센터_보유여부',
    '물품보관함': '물품보관함_보유여부',
    '자전거보관함': '자전거보관함_보유여부',
    '체육시설': '체육시설_보유여부',
    '간이도서관': '간이 도서관_보유여부',
    '쇼핑카트': '쇼핑카트_보유여부',
    '외국인안내센터': '외국인 안내센터_보유여부',
    '고객동선통로': '고객동선통로_보유여부',
    '방송센터': '방송센터_보유여부',
    '문화교실': '문화교실_보유여부',
    '공동물류창고': '공동물류창고_보유여부',
    '시장전용고객주차장': '시장전용 고객주차장_보유여부',
    '교육장': '교육장_보유여부',
    '회의실': '회의실_보유여부',
    '자동심장충격기': '자동심장충격기_보유여부',
    '공중화장실': '공중화장실보유여부',
    '주차장': '주차장보유여부'
}

print(f"📋 최종 컬럼 수: {len(final_columns)}개")


In [ ]:
# 매칭된 데이터 합병
merged_data = []

for facility_idx, std_idx, match_type in matches:
    facility_row = df_facility.loc[facility_idx]
    std_row = df_std.loc[std_idx]
    
    merged_row = {}
    
    # 기본 정보
    merged_row['시장코드'] = facility_row['시장코드']
    merged_row['시장명'] = facility_row['시장명_정제']
    
    # 주소 정보 (첫 번째 파일 우선, 빈값이면 두 번째 파일)
    merged_row['지번주소'] = standardize_region(
        facility_row['지번주소'] if pd.notna(facility_row['지번주소']) and facility_row['지번주소'] 
        else std_row['소재지지번주소'] if pd.notna(std_row['소재지지번주소']) else ''
    )
    
    merged_row['도로명주소'] = standardize_region(
        facility_row['도로명주소'] if pd.notna(facility_row['도로명주소']) and facility_row['도로명주소'] and facility_row['도로명주소'] != '0'
        else std_row['소재지도로명주소'] if pd.notna(std_row['소재지도로명주소']) else ''
    )
    
    # 행정구역 정보
    merged_row['시도'] = standardize_region(
        facility_row['시도'] if pd.notna(facility_row['시도']) and facility_row['시도']
        else ''
    )
    
    merged_row['시군구'] = (
        facility_row['시군구'] if pd.notna(facility_row['시군구']) and facility_row['시군구']
        else ''
    )
    
    # 두 번째 파일에서 가져올 정보
    for col in ['시장개설주기', '점포수', '개설연도', '취급품목', '위도', '경도', '전화번호']:
        merged_row[col] = std_row[col] if pd.notna(std_row[col]) else ''
    
    # 시설 보유여부 정보
    for final_col, original_col in facility_mapping.items():
        if original_col in df_facility.columns:
            merged_row[final_col] = facility_row[original_col] if pd.notna(facility_row[original_col]) else ''
        elif original_col in df_std.columns:
            merged_row[final_col] = std_row[original_col] if pd.notna(std_row[original_col]) else ''
        else:
            merged_row[final_col] = ''
    
    merged_data.append(merged_row)

# DataFrame 생성
df_merged = pd.DataFrame(merged_data, columns=final_columns)

print(f"✅ 매칭된 데이터 합병 완료: {len(df_merged)}개 시장")
print(f"📋 최종 컬럼 수: {len(df_merged.columns)}개")


In [ ]:
# 매칭되지 않은 데이터 분석
print("🔍 매칭되지 않은 데이터 분석...")

# 매칭되지 않은 첫 번째 파일 데이터
matched_facility_indices = set(match[0] for match in matches)
unmatched_facility = df_facility[~df_facility.index.isin(matched_facility_indices)]

# 매칭되지 않은 두 번째 파일 데이터
unmatched_std = df_std[~df_std.index.isin(matched_std_indices)]

print(f"   ⚠️  매칭되지 않은 첫 번째 파일 시장: {len(unmatched_facility)}개")
print(f"   ⚠️  매칭되지 않은 두 번째 파일 시장: {len(unmatched_std)}개")

# 매칭되지 않은 첫 번째 파일 시장들 상세 정보
if len(unmatched_facility) > 0:
    print(f"\n❌ 매칭되지 않은 첫 번째 파일 시장들:")
    for idx, row in unmatched_facility.iterrows():
        print(f"   - {row['시장명_정제']} (코드: {row['시장코드']})")
        print(f"     도로명: {row['도로명주소']}")
        print(f"     지번: {row['지번주소']}")
        print()

# 매칭되지 않은 두 번째 파일 시장들 상세 정보
if len(unmatched_std) > 0:
    print(f"\n❌ 매칭되지 않은 두 번째 파일 시장들:")
    for idx, row in unmatched_std.iterrows():
        print(f"   - {row['시장명_정제']}")
        print(f"     도로명: {row['소재지도로명주소']}")
        print(f"     지번: {row['소재지지번주소']}")
        print()

# 시장명은 같지만 주소가 달라서 매칭 안 된 경우 찾기
print(f"\n🔍 시장명은 같지만 주소 불일치로 매칭 실패한 경우:")
for idx, facility_row in unmatched_facility.iterrows():
    facility_name = facility_row['시장명_정제']
    # 같은 이름의 시장이 두 번째 파일에 있는지 확인
    same_name_std = df_std[df_std['시장명_정제'] == facility_name]
    if len(same_name_std) > 0:
        print(f"\n⚠️  시장명 '{facility_name}' 주소 불일치:")
        print(f"   첫 번째 파일:")
        print(f"     도로명: {facility_row['도로명주소']}")
        print(f"     지번: {facility_row['지번주소']}")
        print(f"   두 번째 파일:")
        for std_idx, std_row in same_name_std.iterrows():
            print(f"     도로명: {std_row['소재지도로명주소']}")
            print(f"     지번: {std_row['소재지지번주소']}")


In [ ]:
# 추가 매칭: 구 정보 차이로 인한 미매칭 해결
print("🔧 추가 매칭: 구 정보 차이 해결...")

def remove_gu_from_address(address):
    """주소에서 구 정보 제거"""
    if pd.isna(address) or not address:
        return ""
    
    address = str(address)
    # 구 정보 패턴 제거 (원미구, 오정구, 소사구 등)
    import re
    address = re.sub(r'\s+[가-힣]+구\s+', ' ', address)
    # 공백 정리
    address = ' '.join(address.split())
    return address

# 매칭되지 않은 데이터들에 대해 구 정보를 제거한 주소로 재매칭 시도
additional_matches = []

# 매칭되지 않은 첫 번째 파일 데이터
matched_facility_indices = set(match[0] for match in matches)
unmatched_facility = df_facility[~df_facility.index.isin(matched_facility_indices)]

# 매칭되지 않은 두 번째 파일 데이터  
unmatched_std = df_std[~df_std.index.isin(matched_std_indices)]

for facility_idx, facility_row in unmatched_facility.iterrows():
    facility_name = facility_row['시장명_정제']
    facility_road_no_gu = remove_gu_from_address(facility_row['도로명주소'])
    facility_lot_no_gu = remove_gu_from_address(facility_row['지번주소'])
    
    # 같은 이름의 미매칭 시장들 중에서 구 정보 제거한 주소로 매칭 시도
    same_name_unmatched = unmatched_std[unmatched_std['시장명_정제'] == facility_name]
    
    for std_idx, std_row in same_name_unmatched.iterrows():
        std_road_no_gu = remove_gu_from_address(std_row['소재지도로명주소'])
        std_lot_no_gu = remove_gu_from_address(std_row['소재지지번주소'])
        
        # 구 정보 제거한 주소가 일치하는지 확인
        road_match = facility_road_no_gu and std_road_no_gu and facility_road_no_gu == std_road_no_gu
        lot_match = facility_lot_no_gu and std_lot_no_gu and facility_lot_no_gu == std_lot_no_gu
        
        if road_match or lot_match:
            additional_matches.append((facility_idx, std_idx, 'address_no_gu'))
            matched_std_indices.add(std_idx)
            print(f"   ✅ 추가 매칭: {facility_name}")
            break

# 기존 매칭에 추가 매칭 결과 합치기
matches.extend(additional_matches)

print(f"   📊 추가 매칭된 시장: {len(additional_matches)}개")
print(f"   📊 총 매칭: {len(matches)}개")


In [ ]:
# 추가 매칭 후 데이터 재병합
print("🔗 추가 매칭 포함 데이터 재병합...")

# 기존 병합 데이터에 추가 매칭 데이터 추가
for facility_idx, std_idx, match_type in additional_matches:
    facility_row = df_facility.loc[facility_idx]
    std_row = df_std.loc[std_idx]
    
    merged_row = {}
    
    # 기본 정보
    merged_row['시장코드'] = facility_row['시장코드']
    merged_row['시장명'] = facility_row['시장명_정제']
    
    # 주소 정보 (첫 번째 파일 우선, 빈값이면 두 번째 파일)
    merged_row['지번주소'] = standardize_region(
        facility_row['지번주소'] if pd.notna(facility_row['지번주소']) and facility_row['지번주소'] 
        else std_row['소재지지번주소'] if pd.notna(std_row['소재지지번주소']) else ''
    )
    
    merged_row['도로명주소'] = standardize_region(
        facility_row['도로명주소'] if pd.notna(facility_row['도로명주소']) and facility_row['도로명주소'] and facility_row['도로명주소'] != '0'
        else std_row['소재지도로명주소'] if pd.notna(std_row['소재지도로명주소']) else ''
    )
    
    # 행정구역 정보
    merged_row['시도'] = standardize_region(
        facility_row['시도'] if pd.notna(facility_row['시도']) and facility_row['시도']
        else ''
    )
    
    merged_row['시군구'] = (
        facility_row['시군구'] if pd.notna(facility_row['시군구']) and facility_row['시군구']
        else ''
    )
    
    # 두 번째 파일에서 가져올 정보
    for col in ['시장개설주기', '점포수', '개설연도', '취급품목', '위도', '경도', '전화번호']:
        merged_row[col] = std_row[col] if pd.notna(std_row[col]) else ''
    
    # 시설 보유여부 정보
    for final_col, original_col in facility_mapping.items():
        if original_col in df_facility.columns:
            merged_row[final_col] = facility_row[original_col] if pd.notna(facility_row[original_col]) else ''
        elif original_col in df_std.columns:
            merged_row[final_col] = std_row[original_col] if pd.notna(std_row[original_col]) else ''
        else:
            merged_row[final_col] = ''
    
    # 기존 DataFrame에 새 행 추가
    df_merged = pd.concat([df_merged, pd.DataFrame([merged_row])], ignore_index=True)

print(f"✅ 재병합 완료: {len(df_merged)}개 시장")
print(f"✅ 최종 매칭률: {len(df_merged)}/{len(df_facility)} = {len(df_merged)/len(df_facility)*100:.1f}%")


In [ ]:
# 결과 저장
print("💾 결과 저장...")

# 매칭된 데이터만 저장
df_merged.to_csv('/content/drive/MyDrive/공공빅데이터분석PBL/midterm/전통시장_매칭데이터.csv', 
                 index=False, encoding='utf-8-sig')

print(f"✅ 매칭 데이터 저장: 전통시장_매칭데이터.csv ({len(df_merged)}개 시장)")

# 데이터 미리보기
print(f"\n🔍 생성된 데이터 미리보기:")
print(df_merged.head())


In [ ]:
# 결과 요약 및 검증
print("📊 최종 결과 요약")
print("=" * 50)

print(f"📁 원본 데이터:")
print(f"   - 첫 번째 파일 (시설 정보): {len(df_facility)}개 시장")
print(f"   - 두 번째 파일 (표준 데이터): {len(df_std)}개 시장")

print(f"\n🔗 매칭 결과:")
print(f"   - 고유 시장명 매칭: {unique_name_matches}개")
print(f"   - 주소 기반 매칭: {address_matches}개")
print(f"   - 총 매칭: {len(matches)}개")
print(f"   - 매칭률: {len(matches)}/{len(df_facility)} = {len(matches)/len(df_facility)*100:.1f}%")

print(f"\n📋 생성된 데이터:")
print(f"   - 매칭된 데이터: {len(df_merged)}개 시장, {len(df_merged.columns)}개 컬럼")

print(f"\n🔍 매칭 예시 (처음 5개):")
for i in range(min(5, len(df_merged))):
    row = df_merged.iloc[i]
    print(f"   {i+1}. {row['시장명']} (코드: {row['시장코드']})")
    addr = str(row['도로명주소'])
    print(f"      주소: {addr[:50]}...") if len(addr) > 50 else print(f"      주소: {addr}")

print(f"\n✨ 특수 처리 확인:")
# '운수대통 생거진천전통시장' 매칭 확인
special_market = df_merged[df_merged['시장명'].str.contains('운수대통.*생거진천', na=False)]
if len(special_market) > 0:
    print(f"   ✅ '운수대통 생거진천전통시장' 매칭 성공: {len(special_market)}개")
    for idx, row in special_market.iterrows():
        print(f"      - {row['시장명']} (코드: {row['시장코드']})")
else:
    print(f"   ⚠️  '운수대통 생거진천전통시장' 매칭 확인 필요")

print(f"\n🎉 전통시장 데이터 합병 완료!")
